# Writing my own fasta to stockholm writter cuz everything sucks
This is what it is supposed to look like https://rfam.org/family/RF00360/alignment.

So I will need to write a 
1. header line
2. sequence ID
3. Sequence
4. Secondary structure
5.end line

In [2]:
def parseFasta(fastafile):
    '''
    Given fasta file return 2 lists
    of headers and sequences.
    
    This can be generated with:
        "clustalo --in=hisps.fa --out=hisps_msa.fa --force"
    '''
    from Bio import SeqIO
    records = SeqIO.parse(fastafile, "fasta")
    
    idList = []
    seqList =[]
    for record in records:
        idList.append(record.id)
        seqList.append(str(record.seq))
        
    return(idList, seqList)

def parseAlignment(alignfile):
    '''
    Given the alignment file produced by
        "RNAalifold --input-format=S hisps_msa.sto > hisps_ss.alifold"
        
    this will return the top alignment!
    '''
    in_file = alignfile
    alignment = ''
    with open(in_file) as vienna_file:
        full_contents = vienna_file.read()
        alignment = full_contents.split('\n')[1].split(' ')[0]
    return alignment

def writeStockholm(ids, seqs, alignment):
    """
    Given the fasta file and secondary structure prediction
    squish them together to look like Rfam's format
    https://rfam.org/family/RF00360/alignment.
    """
    stockholm = ['# STOCKHOLM 1.0\n\n']
    
    idLengths = [len(id) for id in ids]
    position_of_first_char = max( idLengths ) + 7
    num_of_spaces = [position_of_first_char-length for length in idLengths]
    
    for idd, seq, spaces in zip(ids, seqs, num_of_spaces):
        line = f'{idd}{spaces*" "}{seq}\n'
        stockholm.append(line)

    stockholm.append(f'#=GC SS_cons{(position_of_first_char-12)*" "}{alignment}\n//')
    
    return "".join(stockholm)

In [3]:
ids, seqs = parseFasta("hisps_msa.fa")
alignment = parseAlignment("hisps_ss.alifold")
outputfile = "hisps_msa.sto"

with open(outputfile, "w") as file:
    file.write( writeStockholm(ids, seqs, alignment) )

FileNotFoundError: [Errno 2] No such file or directory: 'hisps_msa.fa'